# Multi-Agent Financial Analysis System

This notebook implements a multi-agent system using **Agno** framework with **Google Gemini** models.

## Agents
1. **Financial Document Knowledge Base Expert** - Handles financial sentiment analysis from news
2. **Real-Time CSV Data Agent** - Processes stock price and volume data
3. **Team Leader** - Coordinates responses from both agents

## System Architecture
```
User Query
    ↓
Team Leader Agent (Orchestrator)
    ↓
    ├──→ Financial Document Expert Agent (Gemini 2.0 Flash)
    └──→ CSV Data Agent (Gemini 2.0 Flash)
    ↓
Final Integrated Response
```

## Data Sources
- **Finance Agent**: `financeAgent/data/` - Financial sentiment analysis data
- **CSV Agent**: `csvAgent/data/` - Stock price and volume data


In [63]:
# Install required packages from requirements.txt
import subprocess
import sys

print("Installing required packages from requirements.txt...")
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt", "-q"])

    print("✓ All packages installed successfully!")
except subprocess.CalledProcessError as e:
    print(f"✗ Failed to install packages: {e}")

print("\nInstallation complete!")


Installing required packages from requirements.txt...
✓ All packages installed successfully!

Installation complete!


In [64]:
# Import required libraries
import os
import glob
import pathlib
import pandas as pd
from agno.knowledge.text import TextKnowledgeBase
from agno.knowledge.csv import CSVKnowledgeBase
from agno.agent import Agent

In [65]:
finance_data_dir = pathlib.Path("financeAgent/data")
csv_data_dir = pathlib.Path("csvAgent/data")

finance_data_dir.mkdir(parents=True, exist_ok=True)
csv_data_dir.mkdir(parents=True, exist_ok=True)

In [66]:
finance_kb = TextKnowledgeBase(
    name="finance_kb",
    description="Financial sentiment and document knowledge base",
    path=finance_data_dir  # Specify the path for the knowledge base
)

# Path to the directory containing text files
# finance_data_dir = "path/to/finance_data_dir"
loaded_finance = 0            
# Iterate over all .txt files in the directory
for file_path in glob.glob(os.path.join(finance_data_dir, "*.txt")):
    content = None
    try:
        # Try multiple encodings
        for encoding in ['utf-8', 'latin-1', 'cp1252', 'iso-8859-1']:
            try:
                with open(file_path, "r", encoding=encoding) as file:
                    content = file.read()
                    break
            except UnicodeDecodeError:
                continue
        
        if content:
            # TextKnowledgeBase uses load_documents method with file paths
            finance_kb.load_documents([file_path])
            loaded_finance += 1
            print(f"  ✓ Loaded: {os.path.basename(file_path)}")
        else:
            print(f"  ✗ Warning: Could not decode file {os.path.basename(file_path)}")
    except Exception as e:
        print(f"  ✗ Error loading {os.path.basename(file_path)}: {e}")

INFO Loading knowledge base

WARNING  No vector db provided

  ✓ Loaded: Sentences_50Agree.txt


In [78]:
# Create a new knowledge base for CSV documents
csv_kb = CSVKnowledgeBase(
    name="csv_kb",
    description="Financial CSV knowledge base from CSV documents",
    path=csv_data_dir  # Specify the path for the knowledge base
)

# Path to the directory containing CSV files
# csv_data_dir = "path/to/finance_csv_data_dir"

# Iterate over all .csv files in the directory
for file_path in glob.glob(os.path.join(csv_data_dir, "*.csv")):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    # Convert the DataFrame to a string (or process it as needed)
    content = df.to_string(index=False)  # Convert to a string without row indices
    csv_kb.load_documents(content)

INFO Loading knowledge base

WARNING  No vector db provided

INFO Loading knowledge base

WARNING  No vector db provided

In [81]:
len(content)

201725

In [68]:

from agno.models.google import Gemini

# Define a subclass of Gemini
class CustomGemini(Gemini):
    def parse_provider_response(self, response):
        # Implement the logic to parse the provider's response
        # Replace this with the actual parsing logic
        return {"parsed_response": response}

    def parse_provider_response_delta(self, response_delta):
        # Implement the logic to parse the provider's response delta
        # Replace this with the actual parsing logic
        return {"parsed_response_delta": response_delta}

In [69]:
from agno.models.google import Gemini
finance_agent = Agent(
    name="Finance_Document_Expert",
    role="Financial Document Knowledge Base Specialist",
    model=Gemini(id="gemini-2.5-flash"),
    knowledge=finance_kb,  # Add the knowledge base as a tool
    instructions=[
        "Analyze financial documents thoroughly and provide detailed insights",
        "Reference specific sections of documents when making claims",
        "Provide context and explanations for financial terms and concepts",
        "Be precise with numbers, dates, and financial calculations",
        "When uncertain, clearly state what information is missing",
    ],
    markdown=True,
)

print("✓ Finance Document Expert Agent created successfully!")

✓ Finance Document Expert Agent created successfully!


In [70]:
csv_agent = Agent(
    name="CSV_Data_Analyst",
    role="Real-Time CSV Data Analyst",
    model=Gemini(id="gemini-2.5-flash"),  # Use the subclass here
    knowledge=csv_kb,
    instructions=[
        "Analyze CSV data to extract relevant insights based on queries",
        "Provide statistical summaries and key findings",
        "Highlight trends and patterns in the data",
        "Be specific with numbers and percentages",
        "Mention data quality limitations if present"
    ],
    markdown=True,
)

print("✓ CSV Data Agent created successfully!")


✓ CSV Data Agent created successfully!


In [71]:
# Create Team Leader Agent
team_leader = Agent(
    name="Team_Leader",
    role="Multi-Agent Coordinator and Team Leader",
    model=Gemini(id="gemini-2.5-flash"),
    team=[finance_agent, csv_agent],
    instructions=[
        "Understand the user's query and determine what information is needed",
        "Coordinate with both Finance and CSV agents to gather insights",
        "Synthesize responses from multiple agents into a coherent answer",
        "Highlight synergies and contradictions between document knowledge and real data",
        "Provide a balanced perspective that combines both sources",
        "Clearly structure the final response with sections and proper formatting",
        "If agents provide conflicting information, present both views with context"
    ],
    tools=[finance_agent, csv_agent],  # Add the two agents here
    markdown=True,
)

print("✓ Team Leader Agent created")


✓ Team Leader Agent created


In [72]:
# %pip install agno==1.5.9
def ask_team(query: str):
    try:
        result = team_leader.run(query)
        return result.content
    except Exception as e:
        return f"Error processing query: {str(e)}"


## Usage Examples

The system is now ready to use! Below are some example queries you can try.


In [75]:
# Example 1: Basic query without CSV data
query1 = "can u read the csv files, and print one two rows of the file you can access"

print("Query:", query1)
print("\n" + "="*80 + "\n")

response1 = ask_team(query1)


Query: can u read the csv files, and print one two rows of the file you can access




In [76]:
print(response1)

I apologize, but I am unable to directly access and read files from the file system, including CSV files. My capabilities are limited to the functions provided, and I do not have a function that allows me to list or read local files. Therefore, I cannot identify and extract the first two rows of any accessible CSV files.
